In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import glob
from datetime import datetime as dt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

pd.set_option("display.max_colwidth",999)
pd.set_option("display.max_rows",999)
pd.set_option("display.max_columns",999)

In [2]:
# df_sf_2017 = pickle.load(open('../data_sf_2017.p', 'rb'))

## load 2017 data plus additional manipulations

In [80]:
df_sf_2017 = pd.read_json('df_sf_2017')
df_sf_2017.loc[df_sf_2017.cancellation_policy == 'flexible_new', 'cancellation_policy'] = "flexible"
prop_type = set(df_sf_2017.property_type.values)
keep = ['Apartment', 'House', 'Condomonium']
drop = list(set(prop_type).difference(keep))
df_sf_2017['property_type_new'] = df_sf_2017['property_type']
df_sf_2017.loc[df_sf_2017.property_type.isin(drop), 'property_type_new'] = "Other"

In [54]:
# df_sf['price'].apply(np.log)[df_sf_2017['price_per_guest'].isnull()][[ 'price', 'guests_included','price_per_guest']]

,price,guests_included,price_per_guest


In [81]:
df_sf_2017.price_per_bedroom.isnull().sum()

0

## Random Forest

In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder
import pickle

## Predict 2017

In [101]:
def split_data(start_month, end_month):
    df_sf_temp = df_sf_2017.copy()
    X_train = df_sf_temp[(df_sf_temp['month'] >= start_month) & (df_sf_temp['month'] < end_month)][columns_to_keep]
    y_train = df_sf_temp[(df_sf_temp['month'] >= start_month) & (df_sf_temp['month'] < end_month)]['popular']

    X_test = df_sf_temp[df_sf_temp['month'] == end_month][columns_to_keep]
    y_test = df_sf_temp[df_sf_temp['month'] == end_month]['popular']
    
    return X_train, y_train, X_test, y_test
    

In [109]:
def transform_data(X):
    X['extra_people'].replace('[\=$,]', '', regex=True, inplace=True)
#     X['price'].replace('[\=$,]', '', regex=True, inplace=True)
    X['property_type_new'] = le.fit_transform(X['property_type_new'])
    X['bed_type'] = le.fit_transform(X['bed_type'])
    X['cancellation_policy'] = le.fit_transform(X['cancellation_policy'])
    X['room_type'] = le.fit_transform(X['room_type'])
 
    return X

In [103]:
def predict_data(X_train, y_train, X_test, y_test):
    rf = RandomForestClassifier(n_estimators = 1000, n_jobs=-1, random_state=0, class_weight = {0:.95, 1:.05})
    rf.fit(X_train, y_train)
    predicted = rf.predict(X_test)
    scores[0][model_num] = accuracy_score(y_test, predicted)
    scores[1][model_num] = recall_score(y_test, predicted)
    scores[2][model_num] = precision_score(y_test, predicted)
    scores[3][model_num] = f1_score(y_test, predicted)
    return scores

In [110]:
scores = np.zeros(shape=(4,9))
model_num = 0
start_month = 1
end_month = 4
# columns_to_keep = ['accommodates','bed_type', 'extra_people', 'price','guests_included', 'host_about_filled','host_picture_url_filled', 'cancellation_policy', 'room_type']
# columns_to_keep = ['accommodates','bed_type', 'extra_people', 'price_per_guest', 'price_per_bedroom','guests_included', 'host_about_filled','host_picture_url_filled', 'cancellation_policy', 'room_type']
columns_to_keep = ['access_filled', 'house_rules_filled','space_filled','accommodates','bed_type', 'extra_people', 'price_per_guest', 'price_per_bedroom','guests_included', 'host_about_filled','host_picture_url_filled', 'cancellation_policy', 'room_type', 'property_type_new']

In [111]:
%%time
while end_month <13:
    X_train, y_train, X_test, y_test = split_data(start_month, end_month)
    le = LabelEncoder()
    X_train = transform_data(X_train)
    X_test = transform_data(X_test)
    scores = predict_data(X_train, y_train, X_test, y_test)
    print(start_month, end_month, model_num)
    start_month += 1
    end_month += 1
    model_num += 1
    print(scores)

1 4 0
[[0.92757788 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.68010076 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.8985025  0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.77419355 0.         0.         0.         0.         0.
  0.         0.         0.        ]]
2 5 1
[[0.92757788 0.92533211 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.68010076 0.66162571 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.8985025  0.90128755 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.77419355 0.7630814  0.         0.         0.         0.
  0.         0.         0.        ]]
3 6 2
[[0.92757788 0.92533211 0.92804365 0.         0.         0.
  0.         0.         0.        ]
 [0.68010076 0.66162571 0.68569712 0.         0.         0.
  0.         0.         0.        ]
 [0.8985025  0.90128

In [51]:
from tempfile import TemporaryFile
scores_rf_baseline = TemporaryFile()
np.save(scores_rf_baseline,scores)x

In [53]:
scores_rf_baseline.seek(0) # Only needed here to simulate closing & reopening file
np.load(scores_rf_baseline)

array([[0.81988506, 0.89212093, 0.88964655, 0.88999322, 0.88094975,
        0.88061452, 0.88234636, 0.88732079, 0.85575529],
       [0.2336272 , 0.47006931, 0.48648649, 0.4973822 , 0.47533875,
        0.49067661, 0.49973698, 0.53308824, 0.5445224 ],
       [0.51456311, 0.88075561, 0.87473002, 0.88601036, 0.88855117,
        0.93692777, 0.9047619 , 0.89664311, 0.86982759],
       [0.32135123, 0.61298274, 0.62524122, 0.63710879, 0.61935028,
        0.64405594, 0.64384954, 0.66864295, 0.66976435]])

In [112]:
accuracy_2017 = np.mean(scores[0])
recall_2017 = np.mean(scores[1])
precision_2017 = np.mean(scores[2])
f1_score_2017 = np.mean(scores[3])
print(accuracy_2017)
print(recall_2017)
print(precision_2017)
print(f1_score_2017)

0.9207165118403038
0.6887215870445897
0.9109438933600269
0.784068751449367


In [20]:
df_sf_2017.dtypes == 'object'

access                               True
access_filled                       False
accommodates                        False
amenities                            True
availability_30                     False
availability_365                    False
availability_60                     False
availability_90                     False
bathrooms                           False
bed_type                             True
bedrooms                            False
beds                                False
calculated_host_listings_count      False
calendar_last_scraped               False
calendar_updated                     True
cancellation_policy                  True
city                                 True
cleaning_fee                         True
country                              True
country_code                         True
description                          True
experiences_offered                  True
extra_people                        False
first_review                      

In [18]:
df_sf_2017[columns_to_keep].head()

,access_filled,house_rules_filled,space_filled,accommodates,bed_type,extra_people,price_per_guest,price_per_bedroom,guests_included,host_about_filled,host_picture_url_filled,cancellation_policy,room_type,property_type
100000,True,False,True,3,Real Bed,8,2.406092,4.812184,1,True,True,moderate,Private room,Apartment
100001,False,False,True,2,Real Bed,0,2.561982,5.123964,1,True,True,flexible,Private room,Apartment
100002,True,False,True,5,Real Bed,0,1.221850,3.054624,4,True,True,moderate,Entire home/apt,Apartment
100003,False,False,False,4,Real Bed,0,3.656610,3.656610,1,False,True,flexible,Entire home/apt,Apartment
100004,False,False,False,2,Real Bed,0,2.488367,4.976734,1,True,True,flexible,Private room,House


# Logistic Regression (RF was much better)

In [6]:
from sklearn.linear_model import LogisticRegression

In [13]:
columns_to_keep = ['access_filled', 'house_rules_filled','space_filled','accommodates','bed_type', 'extra_people', 'price_per_guest', 'price_per_bedroom','guests_included', 'host_about_filled','host_picture_url_filled', 'cancellation_policy', 'room_type', 'property_type_new']

In [9]:
def split_data(start_month, end_month):
    df_sf_temp = df_sf_2017.copy()
    X_train = df_sf_temp[(df_sf_temp['month'] >= start_month) & (df_sf_temp['month'] < end_month)][columns_to_keep]
    y_train = df_sf_temp[(df_sf_temp['month'] >= start_month) & (df_sf_temp['month'] < end_month)]['popular']

    X_test = df_sf_temp[df_sf_temp['month'] == end_month][columns_to_keep]
    y_test = df_sf_temp[df_sf_temp['month'] == end_month]['popular']
    
    return X_train, y_train, X_test, y_test
    

In [17]:
def transform_logistic(X, columns_to_keep):

    df_temp = X[columns_to_keep]
    data_dummies = pd.DataFrame([])
    drop = []
    
    #dummify columns
    for column in columns_to_keep:
        if (df_temp[column].dtypes == 'O'):
            drop.append(column)
            dummies = pd.get_dummies(df_temp[column], prefix= column)
            data_dummies = pd.concat([data_dummies, dummies], axis=1)

    #join with original df
    df_temp = df_temp.join(data_dummies)

#     print(list(df_temp.columns.values))
    #drop columns that were dummified
    df_temp.drop(drop, axis=1, inplace = True)

    return df_temp

In [15]:
def predict_logistic(X_train, y_train, X_test, y_test):
    model = LogisticRegression(n_jobs=-1)
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    beta_coef.append(model.coef_)
    scores_logit[0][model_num] = accuracy_score(y_test, prediction)
    scores_logit[1][model_num] = recall_score(y_test, prediction)
    scores_logit[2][model_num] = precision_score(y_test, prediction)
    scores_logit[3][model_num] = f1_score(y_test, prediction)
    return scores_logit, beta_coef

In [21]:
beta_coef = []
scores_logit = np.zeros(shape=(4,9))
model_num = 0
start_month = 1
end_month = 4

In [22]:
%%time
while end_month <13:
    X_train, y_train, X_test, y_test = split_data(start_month, end_month)
    X_train = transform_logistic(X_train, columns_to_keep)
    X_test = transform_logistic(X_test, columns_to_keep)
    scores, beta_coef = predict_logistic(X_train, y_train, X_test, y_test)
    print(start_month, end_month, model_num)
    start_month += 1
    end_month += 1
    model_num += 1
    print(scores_logit)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


1 4 0
[[0.81779515 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.04219144 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.51145038 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.0779523  0.         0.         0.         0.         0.
  0.         0.         0.        ]]
2 5 1
[[0.81779515 0.81768209 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.04219144 0.05229994 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.51145038 0.48538012 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.0779523  0.09442548 0.         0.         0.         0.
  0.         0.         0.        ]]
3 6 2
[[0.81779515 0.81768209 0.81311811 0.         0.         0.
  0.         0.         0.        ]
 [0.04219144 0.05229994 0.08533654 0.         0.         0.
  0.         0.         0.        ]
 [0.51145038 0.48538

In [23]:
accuracy_2017 = np.mean(scores_logit[0])
recall_2017 = np.mean(scores_logit[1])
precision_2017 = np.mean(scores_logit[2])
f1_score_2017 = np.mean(scores_logit[3])
print(accuracy_2017)
print(recall_2017)
print(precision_2017)
print(f1_score_2017)

0.7946514661754123
0.09126554689101034
0.5577129954488265
0.15490514368378314


In [24]:
beta_coef

[array([[ 0.53007962,  0.67561263,  0.15006667,  0.10604996, -0.00633706,
         -1.19853999,  0.18120679, -0.40059964,  0.20662239,  0.55194041,
         -0.62838955, -0.38660583, -0.17662322,  0.09120517,  0.24184857,
         -0.08783601,  0.81886815,  0.49857129, -1.57654837, -0.51161991,
         -0.57295504, -0.21805468, -0.06755512, -0.38932237, -0.08382664,
         -0.38541583]]),
 array([[ 0.4976066 ,  0.61940049,  0.23575569,  0.13464937, -0.00675129,
         -1.5968739 ,  0.19256912, -0.56326032,  0.21077625,  0.62926188,
         -0.63483379, -0.26512533, -0.10449351,  0.24839223,  0.33377135,
         -0.03210413,  0.84458651,  0.566181  , -1.45039404, -0.35055838,
         -0.3713931 , -0.0344765 , -0.01641945, -0.24013341,  0.05059158,
         -0.23274721]]),
 array([[ 0.46440042,  0.62811662,  0.30463579,  0.16218218, -0.00760389,
         -2.00973591,  0.22392864, -0.73386785,  0.24822121,  0.33242678,
         -0.6659505 , -0.10145976, -0.04422087,  0.4338944 ,  

In [14]:
# Compute the change in odds ratio from a one unit change in each feature.

In [143]:
from math import exp
for i, coef in enumerate(model.coef_[0]):
    print("beta%d: %.5f" % (i + 1, exp(coef)))

beta1: 1.69907
beta2: 1.96524
beta3: 1.16191
beta4: 1.11188
beta5: 0.99368
beta6: 0.30163
beta7: 1.19866
beta8: 0.66992
beta9: 1.22952
beta10: 1.73662
beta11: 0.53345
beta12: 0.67936
beta13: 0.83810
beta14: 1.09549
beta15: 1.27360
beta16: 0.91591
beta17: 2.26793
beta18: 1.64637
beta19: 0.20669
beta20: 0.59952
beta21: 0.56386
beta22: 0.80408
beta23: 0.93468
beta24: 0.67752
beta25: 0.91959
beta26: 0.68017


In [ ]:
# Increasing the ____ feature by 1 point increases the chance of being popular by a factor of __ (1.00189).

In [144]:
for i, col in enumerate(list(X_train.columns.values)):
    print(i, col)

0 access_filled
1 house_rules_filled
2 space_filled
3 accommodates
4 extra_people
5 price_per_guest
6 price_per_bedroom
7 guests_included
8 host_about_filled
9 host_picture_url_filled
10 bed_type_Airbed
11 bed_type_Couch
12 bed_type_Futon
13 bed_type_Pull-out Sofa
14 bed_type_Real Bed
15 cancellation_policy_flexible
16 cancellation_policy_moderate
17 cancellation_policy_strict
18 cancellation_policy_super_strict_30
19 cancellation_policy_super_strict_60
20 room_type_Entire home/apt
21 room_type_Private room
22 room_type_Shared room
23 property_type_new_Apartment
24 property_type_new_House
25 property_type_new_Other


## XGBoost

In [107]:
from xgboost import XGBClassifier

In [90]:
df_sf_temp = df_sf_2017.copy()
start_month = 1
end_month = 4
X_train = df_sf_temp[(df_sf_temp['month'] >= start_month) & (df_sf_temp['month'] < end_month)][columns_to_keep]
y_train = df_sf_temp[(df_sf_temp['month'] >= start_month) & (df_sf_temp['month'] < end_month)]['popular']

X_test = df_sf_temp[df_sf_temp['month'] == end_month][columns_to_keep]
y_test = df_sf_temp[df_sf_temp['month'] == end_month]['popular']


In [92]:
X_train = transform_xgboost(X_train, columns_to_keep)

KeyError: "['bed_type' 'cancellation_policy' 'room_type' 'property_type_new'] not in index"

In [93]:
X_test = transform_xgboost(X_test, columns_to_keep)

In [88]:
def transform_xgboost(X, columns_to_keep):
    df_temp = X[columns_to_keep]
    data_dummies = pd.DataFrame([])
    drop = []
    
    #dummify columns
    for column in columns_to_keep:
        if (df_temp[column].dtypes == 'O'):
            drop.append(column)
            dummies = pd.get_dummies(df_temp[column], prefix= column)
            data_dummies = pd.concat([data_dummies, dummies], axis=1)

    #join with original df
    df_temp = df_temp.join(data_dummies)

#     print(list(df_temp.columns.values))
    #drop columns that were dummified
    df_temp.drop(drop, axis=1, inplace = True)
    return df_temp

In [153]:
%%time
my_model = XGBClassifier(n_jobs=-1, booster='gbtree', n_estimators=2000, learning_rate=0.2, objective='binary:logistic')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 26.2 µs


In [154]:
%%time
my_model.fit(X_train, y_train, verbose=False)

CPU times: user 31.7 s, sys: 128 ms, total: 31.9 s
Wall time: 10.6 s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=2000,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [155]:
# Add silent=True to avoid printing out updates with each cycle
predictions = my_model.predict(X_test)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [156]:
accuracy_score(y_test, predictions)

0.8789516036326015

In [157]:
recall_score(y_test, predictions)

0.47418136020151136

In [158]:
precision_score(y_test, predictions)

0.7754891864057673

In [159]:
f1_score(y_test, predictions)

0.5885111371629543

## ADA Boosting

In [160]:
from sklearn.ensemble import AdaBoostClassifier #For Classification

In [162]:
dt = DecisionTreeClassifier() 
clf = AdaBoostClassifier(n_estimators=100, base_estimator=dt,learning_rate=1)
#Above I have used decision tree as a base estimator, you can use any ML learner as base estimator if it ac# cepts sample weight 
clf.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1, n_estimators=100, random_state=None)

In [170]:
prediction = clf.predict(X_test)
accuracy_score(y_test, prediction)

0.9278077939993102

In [169]:
recall_score(y_test, prediction)

0.7594458438287154

In [168]:
precision_score(y_test, prediction)

0.8305785123966942

In [167]:
f1_score(y_test, prediction)

0.7934210526315789

## Gradient Boosting